In [ ]:
"""Урок 3. Транзакции
"""

In [ ]:

from sqlalchemy import Column, Integer, create_engine, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
 
engine = create_engine('sqlite:///flush.db', echo=True)
Session = sessionmaker(bind=engine, autoflush=False)
session = Session()
Base = declarative_base()
 
 
class Parent(Base):
   __tablename__ = 'parent'
   id = Column(Integer, primary_key=True)
   name = Column(String(50), nullable=False)
 
 
if __name__ == '__main__':
   Base.metadata.create_all(engine)
 
   parent = Parent(name="Nikita")
   session.add(parent)
 
   q_1 = session.query(Parent).all()
   print('q1')
   session.commit()
 
   new_session = Session()
   new_session.autoflush = False
 
   new_session.add(Parent(name="Vlad"))
   q_2 = new_session.query(Parent).all()
   print('q2')
   new_session.flush()
 
   q3 = new_session.query(Parent).all()
   print('q3')
 
   # другая сессия не видит
   q4 = session.query(Parent).all()
   print('q4')
 
   new_session.rollback()
 
   q5 = new_session.query(Parent).all()
 
   print('q5')
 
 
   parent = Parent()
   session.add(parent)
   session.flush()
   id = parent.id
   child = Child(parent_id=id)
   session.commit()

"""
Что касается метода begin_nested() - для него flush() вызывается безоговорочно, даже если autoflush = True. 
Для чего, кстати удобно использовать метод flush?
Например, у нас есть две таблицы - родитель и ребенок, где связь один ко многим. Мы создаем родителя, и в этой же транзакции хотим создать детей и привязать их к родителю. ИД родителя - первичный ключ с автоинкрементом, если мы после создания объекта обратимся к данному атрибуту, то получим None, так как запись еще не получила идентификатор. В случае вызова flush() - операция будет выполнения и база данных определит соответствующий идентификатор для родителя. Соответственно, при обращении к атрибуту ИД - мы получим нужный нам ИД, для создания детей и связи с данным родителем. Далее выполним команду commit() и все готово.
В случае, если вы работаете не в декларативном стиле, а напрямую с объектами Engine и Connection работа с транзакциями происходит через объект Connection. Вы можете ознакомиться с документацией, в которой подробно описан способ работы с транзакциями в классическом стиле.
Массовые операции - это менее функциональные версии ORM-инструкций INSERT или UPDATE. Они созданы для того, чтобы сократить накладные расходы ORM, а именно:
Метод Session.flush(), каскадное поведение, статус всех связанных через relationship() объектов полностью игнорируется.
 Это значительно снижает расходы на уровне Python.
Объекты, участвующие в массовых операциях не имеют связи с сессией. 
Это означает, что даже когда операция завершена, состояние сессии не будет обновляться, отсутствует очистка объект из сессии.
Операции вставки и изменения принимают списки простых словарей, а не объекты.
Это дополнительно снижает большие накладные расходы, связанные с созданием экземпляров объектов и назначением им состояния.
Процесс получения первичных ключей после операции INSERT отключен. 
Операторы INSERT будут объединены в executemany() блоки, которые работают намного быстрее, чем вызовы отдельных операторов для каждой строки.
Для оператора UPDATE, можно настроить, чтобы все атрибуты словаря попадали под действие конструкции SET, что повышает вероятность использования executemany() блоков.
Массовые операции включают в себя три метода Session.bulk_save_objects(), Session.bulk_insert_mappings(), and Session.bulk_update_mappings().
bulk_save_objects(objects, *args, **kwargs) - массовое сохранение списка объектов. Функция позволяет использовать операции  INSERT и UPDATE для группы входных объектов. 
То есть у нас есть список сгруппированных объектов, где часть из них должны вызвать INSERT, а часть UPDATE. В таком случае будут вызваны два запроса: один на добавление, другой на изменение.
bulk_insert_mappings(mapper , mappings, *args, **kwargs) - массовая вставка входного списка словарей.
Здесь, как вы поняли мы работаем не с экземплярами таблицы, а с обычным список словарей. Обязательными параметрами являются mapper, в декларативной стиле сам ORM-объект и список словарей. Значения в словарях обычно передаются без изменений в специальную конструкцию insert(), про нее мы разговаривали в предыдущем модуле.
bulk_update_mappings (mapper , mappings) - массовое обновление входного списка словарей
Аналогично вставке на вход функция ждет ORM-объект или mapper и список словарей с изменениями. При этом словарь должен содержать первичный ключ, по которому будет отрабатывать конструкция WHERE.
""" 
from sqlalchemy import Column, Integer, create_engine, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
 
engine = create_engine('sqlite:///mass_operations.db', echo=True)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()
 
 
class Parent(Base):
   __tablename__ = 'parent'
   id = Column(Integer, primary_key=True)
   name = Column(String(50), nullable=False)
 
 
if __name__ == '__main__':
   Base.metadata.create_all(engine)
 
   # bulk_save_objects
   parent_1 = Parent(name="Nikita")
   parent_2 = Parent(name="Nastya")
   parent_3 = Parent(name="Vlad")
   parent_4 = Parent(name="Lera")
 
   session.bulk_save_objects([parent_1, parent_2, parent_3, parent_4])
   session.commit()
 
   parents = session.query(Parent).all()
 
   # bulk_insert_mappings
   insert_parents = [
       {"name": "Nikita2"},
       {"name": "Nastya2"},
       {"name": "Vlad2"},
       {"name": "Lera2"},
   ]
   session.bulk_insert_mappings(Parent, insert_parents)
   session.commit()
 
   parents = session.query(Parent).all()
 
   # bulk_update_mappings
   update_parents = [
       {"id": 1, "name": "Nikita_new"},
       {"id": 2, "name": "Nastya_new"},
   ]
   session.bulk_update_mappings(Parent, update_parents)
   session.commit()
   parents = session.query(Parent).all()

